In [23]:
import pandas as pd
import numpy as np

In [33]:
def process_excel_file(file_path, name_list, output_path):
    """
    处理Excel文件，根据规则填写稽核结果和备注列
    优先级：规则一 > 规则二 > 规则三

    参数:
    file_path: Excel文件路径
    name_list: 集团名称列表
    output_path: 输出文件路径
    """
    # 读取Excel文件
    df = pd.read_excel(file_path)

    # 初始化稽核结果和备注列为空
    df['稽核结果'] = np.nan
    df['备注'] = ''

    # 创建一个标记列，用于记录哪些行已经被处理过
    df['已处理'] = False

    # 规则1: 如果集团名称在name_list中（最高优先级）
    rule1_mask = df['集团名称'].isin(name_list)
    df.loc[rule1_mask, '稽核结果'] = 0
    df.loc[rule1_mask, '备注'] = '卡+模组'
    df.loc[rule1_mask, '已处理'] = True

    # 规则2: 如果奖金结果小于1（次优先级，只处理未被规则1处理的行）
    rule2_mask = (df['奖金结果'] < 1) & (~df['已处理'])
    df.loc[rule2_mask, '稽核结果'] = df.loc[rule2_mask, '奖金结果']
    df.loc[rule2_mask, '已处理'] = True

    # 规则3: 处理应收费用连续相同的情况（最低优先级，只处理未被前两个规则处理的行）
    # 首先按集团名称分组
    for group_name, group_df in df.groupby('集团名称'):
        # 只处理该组中未被处理的行的索引
        group_unprocessed_indices = group_df[~group_df['已处理']].index
        if len(group_unprocessed_indices) == 0:
            continue

        # 创建临时DataFrame用于排序处理
        temp_df = df.loc[group_unprocessed_indices].copy()
        temp_df['原始索引'] = group_unprocessed_indices

        # 对应收费用进行升序排序，但保留原始索引信息
        temp_df_sorted = temp_df.sort_values('应收费用').reset_index(drop=True)

        # 处理排序后的数据
        receivable_fees = temp_df_sorted['应收费用'].values
        n = len(receivable_fees)

        i = 0
        while i < n:
            # 查找连续相同的应收费用值
            j = i + 1
            while j < n and receivable_fees[j] == receivable_fees[i]:
                j += 1

            # 连续相同值的数量
            consecutive_count = j - i

            if consecutive_count > 1:  # 有连续相同值
                x = receivable_fees[i]

                # 检查是否有下一个不同的值
                if j < n:
                    expected_value = x + consecutive_count + 1

                    if receivable_fees[j] == expected_value:
                        # 设置连续相同值的稽核结果
                        for k in range(i, j):
                            original_idx = temp_df_sorted.iloc[k]['原始索引']
                            if not df.at[original_idx, '已处理']:
                                df.at[original_idx, '稽核结果'] = df.at[original_idx, '奖金结果']
                                df.at[original_idx, '已处理'] = True

                        # 设置下一个不同值的稽核结果和备注
                        next_original_idx = temp_df_sorted.iloc[j]['原始索引']
                        if not df.at[next_original_idx, '已处理']:
                            audit_result = (x + (consecutive_count + 1) / 12) / 2
                            df.at[next_original_idx, '稽核结果'] = audit_result
                            df.at[next_original_idx, '备注'] = f'卡费{consecutive_count + 1}元一次性出账'
                            df.at[next_original_idx, '已处理'] = True

                i = j  # 移动到下一个不同的值
            else:
                i += 1

    # 删除临时列
    df.drop('已处理', axis=1, inplace=True)

    # 保存处理后的文件
    df.to_excel(output_path, index=False)
    print(f"处理完成，结果已保存到: {output_path}")
    print(f"总共处理了 {len(df)} 行数据")

    # 显示各规则处理的行数统计
    rule1_count = rule1_mask.sum()
    rule2_count = rule2_mask.sum()
    rule3_count = len(df[~df['稽核结果'].isna()]) - rule1_count - rule2_count
    print(f"规则一处理: {rule1_count} 行")
    print(f"规则二处理: {rule2_count} 行")
    print(f"规则三处理: {rule3_count} 行")
    print(f"未处理: {df['稽核结果'].isna().sum()} 行")



In [34]:

if __name__ == "__main__":
    # 你的集团名称列表
    name_list = ["江苏丰景信息技术有限公司", "浙江智电生态科技有限公司", "南通跃鸿信息科技有限公司", "浙江瀚达环境科技有限公司(嘉兴)", "安徽即刻智能科技有限公司","杭州多协信息技术有限公司","浙江和达科技股份有限公司","杭州瀚联传感器技术有限公司（嘉兴）"]  #

    # 输入文件路径和输出文件路径
    input_file = "F:/嘉兴移动/政企/物联网/量化薪酬/8月/【已核对】202508CMIOT清单稽核.xlsx"  #
    output_file = "F:/嘉兴移动/政企/物联网/量化薪酬/8月/output.xlsx"  #

    # 处理文件
    process_excel_file(input_file, name_list, output_file)

处理完成，结果已保存到: F:/嘉兴移动/政企/物联网/量化薪酬/8月/output.xlsx
总共处理了 190654 行数据
规则一处理: 83669 行
规则二处理: 6389 行
规则三处理: 85 行
未处理: 100511 行


In [10]:
df = pd.read_excel("F:/嘉兴移动/政企/物联网/量化薪酬/8月/202507CMIOT稽核清单.xlsx" )

In [18]:
df['已处理'] = False
~df['已处理'][0]

np.True_

In [ ]:
rule2_mask = (df['奖金结果'] < 1) & (~df['已处理'])
df.loc[rule2_mask, '稽核结果'] = df.loc[rule2_mask, '奖金结果']
df.loc[rule2_mask, '已处理'] = True